In [170]:
# !pip install yargy

In [171]:
# !pip install ipymarkup

In [172]:
import gzip
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import eq, in_, type, normalized, dictionary, gte, lte
from yargy.interpretation import fact
from yargy.predicates import gram, is_capitalized
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline
from random import seed, sample
from ipymarkup import show_span_ascii_markup as show_markup
from dataclasses import dataclass
from typing import Iterator


In [173]:
@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("../Data/news.txt.gz"))

In [174]:

def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
     

In [175]:

seed(3)
for text in sample(texts, 3):
    print(text)
    print('---' * 10)

Text(label='media', title='Вещатели назвали переоцененным влияние Солнца на телесигнал', text='В ФГУП «Российская телевизионная радиовещательная сеть» (РТРС) опровергли информацию о том, что 8 марта в ряде городов страны вещание прервется из-за повышенной солнечной активности. Об этом говорится в комментарии компании, поступившем в редакцию «Ленты.ру».Таким образом в РТРС прокомментировали появившуюся в СМИ информацию о том, что из-за солнечной интерференции (явление, когда Солнце, спутник связи и приемная антенна наземного ретранслятора выстраиваются на одной линии) на экранах телевизоров в Международный женский день могут возникнуть значительные помехи, а где-то сигнал может и вовсе пропасть.«Солнце — мощный источник радиосигнала — оказавшись ровно позади спутника связи, также "передает" свой сигнал наземному ретранслятору. Этот сигнал способен вызвать кратковременные — не более пяти минут — ухудшения качества приема в отдельных районах страны. На телевизионных экранах могут появлять

In [ ]:
Name = fact(
    'Name',
    ['first', 'last'],
)

gnc = gnc_relation()

LAST = and_(
    gram('Surn'),
    not_(gram('Abbr')),
    is_capitalized(),
).interpretation(
    Name.last.inflected()
).match(gnc)

FIRST = and_(
    gram('Name'),
    not_(gram('Abbr')),
    is_capitalized(),
).interpretation(
    Name.first.inflected()
).match(gnc)


NAME = or_(
    rule(
        FIRST,
        LAST
    ), 
    rule(
        LAST,
        FIRST
    ),
        rule(
        LAST,
    ),
    rule(
        FIRST,
    ),
).interpretation(
    Name
)


In [177]:

print("Name")
show_matches(
    NAME,
    'управляющий директор Иван Ульянов',
    'Иванов Андрей играет на банжо',
    'Виктор Андреевич Сухоруков развивает навыки скоростного чтения',
    'Сергей отдыхает в Анапе',

)

Name
управляющий директор Иван Ульянов
                     ────────────
Иванов Андрей играет на банжо
─────────────                
Виктор Андреевич Сухоруков развивает навыки скоростного чтения
──────           ─────────                                    
Сергей отдыхает в Анапе
──────                 


In [178]:
Entry = fact(
    'Entry',
    ['name', 'birth_date', 'birth_place']
)

Date = fact(
    'Date',
    ['year', 'month', 'day']
)

gnc = gnc_relation()


Birth_place = fact(
    'Birth_place',
    ['birth_place']
)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1700),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = or_(
    rule(
        DAY,
        '.',
        MONTH_NAME,
        '.',
        YEAR
    ),
    rule(
        gram("PREP").optional(),
        DAY.optional(),
        MONTH_NAME.optional(),
        YEAR,
        normalized('год').optional()
    )
).interpretation(
    Date
)

PLACE_TAG = morph_pipeline([
    "город", 
    "село",
    "поселок",
    "деревня",
])

In [179]:

BIRTH_PLACE = rule(
    gram('PREP'),
    PLACE_TAG.optional(),
    and_(
        is_capitalized(),
        gram("NOUN")
    ).interpretation(Birth_place.birth_place.inflected())
).interpretation(Birth_place)

print("Date")
show_matches(
    DATE,
    'родился в городе Москва 21 февраля 1990 года',
    'родиться 32 сентября 2000 ',
    'родилась 01 июль 1917 года',
    '12 декабря 2012',
)

print("Birth place")
show_matches(
    BIRTH_PLACE,
    'родился в городе Москва 21 февраля 1990 года ',
    'родиться 32 сентября 2000 в семье ученых',
    'родилась 01 июль 1917 года в Шабаново',
    '12 декабря 2012 в Томске',
)

Date
родился в городе Москва 21 февраля 1990 года
                        ────────────────────
родиться 32 сентября 2000 
            ───────────── 
родилась 01 июль 1917 года
         ─────────────────
12 декабря 2012
───────────────
Birth place
родился в городе Москва 21 февраля 1990 года 
        ───────────────                      
родиться 32 сентября 2000 в семье ученых
родилась 01 июль 1917 года в Шабаново
                           ──────────
12 декабря 2012 в Томске
                ────────


In [180]:
ENTRY = or_(
    rule(
        NAME.interpretation(
            Entry.name
        ),
        normalized('родиться').optional(),
        DATE.interpretation(
            Entry.birth_date
        ),
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        ),    
    ),
    rule(
        NAME.interpretation(
            Entry.name
        ),
        normalized('родиться').optional(),
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        ),
        DATE.interpretation(
            Entry.birth_date
        ),
    ),
    rule(
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        ),
        NAME.interpretation(
            Entry.name
        ),
        normalized('родиться').optional(),
        DATE.interpretation(
            Entry.birth_date
        ),
    ),
    rule(
        BIRTH_PLACE.interpretation(
            Entry.birth_place
        ),
        normalized('родиться').optional(),
        NAME.interpretation(
            Entry.name
        ),
        DATE.interpretation(
            Entry.birth_date
        ),
    ),
).interpretation(
    Entry
)

In [ ]:
print("Name")
show_matches(
    NAME,
    'Иван Ульянов родился 30 ноября 1960 года в Саратовe',
)

print("Date")
show_matches(
    DATE,
    'Иван Ульянов родился 25 ноября 1960 года в Саратовe',
)

print("Birth Place")
show_matches(
    BIRTH_PLACE,
    'Иван Ульянов родился 30 ноября 1960 года в городе Саратов',
)

print("Entry")
show_matches(
    ENTRY,
    'Писатель Иван Ульянов родился в Саратове 25 ноября 1960 года в семье ученых',
)

parser = Parser(ENTRY)

match = parser.match('Иван Ульянов родился 30 ноября 1960 года в городе Саратов')
print(match)

Name
Иван Ульянов родился 30 ноября 1960 года в Саратовe
────────────                                       
Date
Иван Ульянов родился 25 ноября 1960 года в Саратовe
                     ───────────────────           
Birth Place
Иван Ульянов родился 30 ноября 1960 года в городе Саратов
                                         ────────────────
Entry
Писатель Иван Ульянов родился в Саратове 25 ноября 1960 года в семье 
         ───────────────────────────────────────────────────         
ученых
Match(tokens=[MorphToken(value='Иван', span=[0, 4), type='RU', forms=[Form('иван', Grams(NOUN,Name,anim,masc,nomn,sing))]), MorphToken(value='Ульянов', span=[5, 12), type='RU', forms=[Form('ульянов', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))]), MorphToken(value='родился', span=[13, 20), type='RU', forms=[Form('родиться', Grams(VERB,indc,intr,masc,past,perf,sing)), Form('родиться', Grams(VERB,impf,indc,intr,masc,past,sing))]), Token(value='30', span=[21, 23), type='INT'), MorphToken(value='ноября

In [182]:
text = texts[2514].text
for match in parser.findall(text):
    result = match.fact
    print(text), print(result), print("\n")


Президент России Владимир Путин и премьер-министр Дмитрий Медведев поздравили с 70-летием актера театра и кино, народного артиста РСФСР Николая Караченцова. Телеграммы опубликованы на официальном сайте главы государства, а также на сайте правительства.Владимир Путин отметил, что юбилей артиста — «большое событие для всех почитателей щедрого, самобытного дарования». «С именем Николая Караченцова связаны яркие, по-настоящему звездные страницы истории отечественного драматического искусства, блистательная плеяда незабываемых образов, которые Вы создали в кино и на сцене легендарного "Ленкома"», — отметил в телеграмме глава государства.Дмитрий Медведев, в свою очередь, подчеркнул, что актера «знают и любят миллионы зрителей нашей страны». «Мастерское владение профессией, блистательная импровизация и неповторимый голос всегда помогали Вам играть ярко и вдохновенно, создавать глубокие образы в театре и в кино, быть органичным в любом жанре — драме и комедии, мюзикле и рок-опере», — написал п

In [183]:
parser = Parser(ENTRY)

seed(7)
for text in sample(texts, 100):
    matches = list(parser.findall(text.text))
    if matches:
        match = matches[0]
        fact = match.fact
        # show_markup(str(text), fact.spans)
        print(text), print(fact), print("\n")

Text(label='media', title='Московский репортер The Guardian обвинила Reuters в плагиате', text='Мириам Элдер (Miriam Elder), московский корреспондент The Guardian, в своем аккаунте в Twitter обвинила Reuters в копировании ее репортажа про Чечню.30 сентября Элдер опубликовала на сайте The Guardian статью про книгу Полины Жеребцовой о войне в Чечне. В записи от 27 октября Элдер дала ссылку на статью Reuters на ту же тему и добавила: "Вау. Спасибо Reuters за то, что поменяли хотя бы несколько слов в моей статье". Статья московского корреспондента Reuters Алисы де Карбоннел (Alissa de Carbonnel) появилась на сайте издания 27 октября и на данный момент недоступна. В одной из следующих записей в Twitter Элдер сообщила, что они с Карбоннел дружат и что "тем более странной выглядит вся эта история".Как пишет американский сайт AdWeek, несмотря на то, что структура и язык обеих статей были очень похожи, никаких официальных обвинений в плагиате предъявлено не было. Как выяснилось, и Элдер, и Карб